# base data

In [ ]:
# =============================================================================
# download missing bchusdt 1m candle data and insert into sqlite (utc + ms)
# =============================================================================
import os
import json
import sqlite3
import pandas as pd
from binance.client import Client
from datetime import datetime, timedelta, timezone

# =============================================================================
# load api keys
# =============================================================================
keys_path  = "C:/connection/binance_keys.json"
with open(keys_path, "r") as f:
    keys     = json.load(f)
API_KEY     = keys["api_key"]
API_SECRET  = keys["api_secret"]

# =============================================================================
# initialize binance client
# =============================================================================
client      = Client(API_KEY, API_SECRET)

# =============================================================================
# database/table params
# =============================================================================
db_path     = "data/bchusdt_data.db"
table       = "bchusdt_1m"
symbol      = "BCHUSDT"
interval    = Client.KLINE_INTERVAL_1MINUTE
os.makedirs("data", exist_ok=True)

# =============================================================================
# determine start_ms from db (use original binance ms), fallback to 2017-01-01
# =============================================================================
conn        = sqlite3.connect(db_path)
cursor      = conn.cursor()
cursor.execute(f"SELECT MAX(open_time_ms) FROM {table}")
row         = cursor.fetchone()
conn.close()

if row is None or row[0] is None:
    # 2017-01-01 00:00:00 utc in milliseconds
    start_ms = int(datetime(2017, 1, 1, 0, 0, 0, tzinfo=timezone.utc).timestamp() * 1000)
else:
    # next minute after the last stored candle
    start_ms = int(row[0]) + 60_000

# =============================================================================
# compute end_ms from binance server time (floor to newest complete minute)
# =============================================================================
server_ms   = client.get_server_time()["serverTime"]      # binance server utc ms
end_ms      = server_ms - (server_ms % 60_000)            # floor to full minute

# guard: nothing to download
if start_ms >= end_ms:
    print("up-to-date: no new minutes to download")
    raise SystemExit(0)

# optional logging
start_dt    = datetime.fromtimestamp(start_ms / 1000, tz=timezone.utc)
end_dt      = datetime.fromtimestamp(end_ms   / 1000, tz=timezone.utc)
print("download window (utc):")
print("  start:", start_dt.isoformat().replace("+00:00", "Z"), f"({start_ms})")
print("  end  :", end_dt.isoformat().replace("+00:00", "Z"), f"({end_ms})")

OperationalError: no such column: open_time_ms

In [ ]:

# =============================================================================
# fetch historical klines using millisecond timestamps
# =============================================================================
raw_data    = client.get_historical_klines(
    symbol,
    interval,
    start_ms,
    end_ms
)

if not raw_data:
    print("binance returned no rows for the given window")
    raise SystemExit(0)

# =============================================================================
# transform to dataframe; keep original ms and add utc iso text
# =============================================================================
df          = pd.DataFrame(
    raw_data,
    columns=[
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base", "taker_buy_quote", "ignore"
    ]
)

# keep original open_time as ms for alignment with binance
df["open_time_ms"] = df["open_time"].astype("int64")

# convert to utc iso string (minute-aligned). e.g., '2025-10-22T12:46:00Z'
df["open_time"]    = pd.to_datetime(df["open_time_ms"], unit="ms", utc=True) \
                        .dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# numeric columns
for c in ["open", "high", "low", "close", "volume"]:
    df[c] = df[c].astype(float)

# select final columns
df = df[["open_time_ms", "open_time", "open", "high", "low", "close", "volume"]]

# =============================================================================
# insert into sqlite with upsert behavior (ignore duplicates by primary key)
# =============================================================================
conn        = sqlite3.connect(db_path)
cursor      = conn.cursor()

records     = list(df.itertuples(index=False, name=None))
cursor.executemany(
    f"""
    INSERT OR IGNORE INTO {table}
    (open_time_ms, open_time, open, high, low, close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """,
    records
)

conn.commit()
conn.close()

print(f"inserted rows: {len(records)} (duplicates, if any, were ignored)")
print(f"range inserted: {df['open_time'].min()} -> {df['open_time'].max()}")